# MACROS
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents
    
* Overview selected macro features in the following format:
 * feature description
 * where is it implemented in the code
 * simple example
 * real use case

## Contents

The selectd features are:
 * Asynchronous Motion and Acquisition
 * Repeat Parameters
 * GSF: Generic Scan Framework
 * Execute submacro and get data
 * Hooks
 * Custom Recorders

## MACROS

# Asynchronous Motion and Acquisition

### Macros: Asynchronous Motion and Acquisition

### Feature Description

* From a macro, or any other code using the Sardana-Taurus extension allow:
 * To launch a motion/acquisition
 * To perform an action while the motion/acquisition is still happening (asynchronous)
 * To perform some other action once the motion/acquisition has finished (synchronous with the end of the motion/acquisition)

### Macros: Asynchronous Motion and Acquisition

### Where in the code?

* Synchronization with the end of a motion or acquisition has finished. Waiting till the motion or acqusition has finished: <br>
[`sardana/taurus/core/tango/sardana/pool.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/taurus/core/tango/sardana/pool.py) <br>
(class **PoolElement**, method **waitMove**)

### Macros: Asynchronous Motion and Acquisition

### Simple Example

An example of a motion waiting for the end of the motion in order to synchronize other actions with the end of the acquisition can be found in the macro **move_asynch** of the following module: [`sardana/macroserver/macros/examples/motion.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/motion.py)

**Description of the macro move_asynch**
* The initial position is displayed
* The motion starts
* A reading of the State of the motion is done asynchronously
* A wait is present in order to wait to the end of the motion
* Synchronously with the end of the motion a readPosition is done




### Macros: Asynchronous Motion and Acquisition

### Real Application

* **macro mar_ct**: [`ALBA_BL04_MSPD/hpexp.py`](https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL04_MSPD/hpexp.py)
* BL04 @ ALBA


* Description of the synchronization interesting macro steps:
 * The acquisition begins
 * The Detector is monitored
 * A wait for the measurement group to finish the acquisition is used
 * Synchronized with the end of the acquisition, the header is populated, and the Image is saved

## MACROS


# Repeat Parameters

### Macros: Repeat Parameters

### Feature Description

* Features of parameter repetion in macros:
 * Multiple repetitions of a single parameter are allowed
 * Multiple repetitions of groups of parameters are allowed
 * Multiple repetitions of nested groups of parameters are allowed
 * Parameters in all positions can be repeated
 * Spock: new interface for parameter repetition using square brackets
 * Spock: default value used if no element present inside the square brakets

### Macros: Repeat Parameters

### Where in the code?


* Spock parsing of parameters:

[`sardana/spock/parser.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/spock/parser.py) (class **ParamParser**)

[`sardana/spock/spockms.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/spock/spockms.py) (function **split_macro_parameters**)


* RepeatParamNode and RepeatNode:

[`sardana/taurus/core/tango/sardana/macro.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/taurus/core/tango/sardana/macro.py) (class **RepeatParamNode**)

[`sardana/taurus/core/tango/sardana/macroserver.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/taurus/core/tango/sardana/macroserver.py) (method **validateRepeatParam**)


* Macroexecutor: Duplicate action

[`sardana/taurus/qt/qtgui/extra_macroexecutor/macroparameterseditor/macroparameterseditor.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/taurus/qt/qtgui/extra_macroexecutor/macroparameterseditor/macroparameterseditor.py) (QAction **duplicateAction**)

### Macros: Repeat Parameters

### Simple Example


Simple examples of Repeat Parameters usage can be found on the macro examples. Many examples presenting Repeat Parameters are also present in this Sardana python module.

[`sardana/macroserver/macros/examples/parameters.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/parameters.py)

---

* Example: macro pt7 presents a **repeat parameter** containing a motor and a position for each repetition:

In [3]:
import mock
Type = mock.MagicMock()

from sardana.macroserver.macro import Macro

class pt7(Macro):
    """Macro with a list of pair Motor,Float.
    Usages from Spock, ex.:
    pt7 [[mot1 1] [mot2 3]]
    pt7 mot1 1 mot2 3
    """

    param_def = [
        ['m_p_pair', [['motor', Type.Motor, None, 'Motor to move'],
                      ['pos',   Type.Float, None, 'Position to move to']],
         None, 'List of motor/position pairs']
    ]

    def run(self, *args, **kwargs):
        pass

### Macros: Repeat Parameters

### Real Application

* BL09 @ ALBA

* Requirements 
 * Generation of collect script for TXM Microscope 
 * Many groups of parameters with few changes in numeric values
 * Collection of images for: many lenses position - many angles - many energies 

* Solution: 
 * Macro **manytomos** launched from macroexecutor
 * Usage of nested repeat parameters
 * Macroexecutor group duplication option

**manytomos** macro in MacroExecutor

![alt text](manytomos.jpeg "Repeat Parameters and Duplicate Group in Macroexecutor")


## MACROS

# GSF: Generic Scan Framework

### Macros: Generic Scan Framework (GSF)

### Feature Description

* The generic scan framework allows to generate custom made scans
* It allow to generate arbitrary number of points, regions, etc. 
* It allows to generate scans with heterogeneous patterns in scan points and integration time
* The rest of the scans, inherits from the this **GScan** generic scan.

### Macros: Generic Scan Framework (GSF)

### Where in the code?


 * In class **GScan** from scan.py
 [`sardana.macroserver.scan.gscan.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/scan/gscan.py)

### Macros: Generic Scan Framework (GSF)

### Simple Example

The following command executes a region scan in spock:
*regscan movable integration_time start_position [[next_position  n_intervals]]*

```
%regscan dummymotor01 0.1 2 [[3 5] [10 7]]
```

* The presented scan:
 * Has an integration time of 0.1 seconds
 * It begins at position 2.
 * It has 5 intervals between position 2 and position 3
 * It has 7 intervals between position 3 and position 10
 * A total of 13 points are taken
 
 
* Macro **regscan**
 [`sardana/macroserver/macros/examples/scans.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/scans.py) 

### Macros: Generic Scan Framework (GSF)

### Real Application

* BL22 @ ALBA



* **Macro constKscan:**
This Macro performs an Energy Scan using the GSF with NON equidistant distances in each of the intervals. The objective is to get an absorption spectrum as output. The scan collects more points near to the spectrum peaks and it increases the integration time as the energy increases.



* The **constKscan** code can be found here:
[`ALBA_BL22_CLAESS/bl22scans.py`](https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL22_CLAESS/bl22scans.py)


## MACROS

# Hooks


### Macros: Hooks

### Feature Description


* Hooks are used to insert code at specific locations within the macro
* The following kind of hooks exists for the scans:
 * pre-scan
 * pre-move
 * post-move
 * pre-acq
 * post-acq
 * post-step
 * post-scan
* Customized hooks can be created

### Macros: Hooks

### Where in the code?


 * In class **Hookable** from macro.py; method: **hooks**:
  * [`sardana.macroserver.macros.macro.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/macro.py)
 
 
 
 
 * Hooks defined along the scans:
  * [`sardana.macroserver.scan.gscan.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/scan/gscan.py)

### Macros: Hooks

### Simple Example

* The macro **hooked_scan** from the Sardana macro examples, shows how hooks can be included in different locations of the macro execution:
 * [`sardana/macroserver/macros/examples/hooks.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/hooks.py)
 

* More examples can also be found in the same python module  [`sardana/macroserver/macros/examples/hooks.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/hooks.py)

### Macros: Hooks

### Real Application

* BL22 @ ALBA

* Macro **qExafs** which inherits from class **BL22qExafs**. In method **run_scan**

* [`ALBA_BL22_CLAESS/bl22scans.py`](https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL22_CLAESS/bl22scans.py)

* **qExafs** macro uses many different scan hooks:
 * pre_configure_hook
 * post_configure_hook
 * pre_start_hook
 * post_move_hook
 * cleanup


 


## MACROS

# Execute submacro and get data

### Macros: Execute submacro and get data

### Feature Description

* Requirement:
 * Obtain the data from a macro
 * Use this data in another macro

* How to satisfy the requirement:
 * Macro can expose its data with the `data` property
 * Clients can obtain the data from the Door
 * The data of the last executed macro is available: attribute *RecordData* of the Door
 



### Macros: Execute submacro and get data

### Where in the code?

* From a macro, the following methods can be used to execute another macro, and get its data:

 * In class **Macro** from macro.py: methods **execMacro**, **getData** and **getResult**
 [`sardana.macroserver.macro.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macro.py)

### Macros: Custom Recorders

### Simple Example



* The macro **get_data**, shows a basic example of:
 * Executing a macro from within another macro
 * Get its data
 * Set the result: In this case we calculate the middle position between the motor end position and the motor start position.


[`sardana/macroserver/recorders/examples/submacros.py`](https://github.com/sardana-org/sardana/blob/develop/src/sardana/macroserver/macros/examples/submacros.py) (Macro **get_data**)




### Macros: Execute submacro and get data


### Real Application

* BL01 @ ALBA
 * Beamline mirror alignment in order to get the maximum intensity in the microscope
 * Rocking curve fitting and go to peak


* Macros involved:
 * **Perform a scan** by moving the a motor of the matching unit
 * **get_peak Macro:** Perform a Gaussian fit with the data of the scan: 
 * **go_peak Macro:** Go to the peak of the Gaussian curve by getting the output of get_peak macro

## MACROS

# Custom Recorders

### Macros: Custom Recorders

### Feature Description

* Create Custom Recorders
* Indicate the custom recorder to be used with one of the following options:
  1. By setting the Spock variable [**`ScanRecorder`**](http://www.sardana-controls.org/en/latest/users/scan.html#configuration). Indicate the recorder path in the MacroServer property **RecorderPath**
  2. By defining the dictionary [**`SCAN_RECORDER_MAP`**](http://www.sardana-controls.org/en/latest/devel/howto_recorders.html#configuration) in sardanacustomsettings

### Macros: Custom Recorders

### Where in the code?


 * The Sardana code for the data recorders can be found in:
  * [`sardana.macroserver.scan.recorder`](https://github.com/sardana-org/sardana/tree/develop/src/sardana/macroserver/scan/recorder)
  * [`sardana.macroserver.recorders`](https://github.com/sardana-org/sardana/tree/develop/src/sardana/macroserver/recorders)
 
 * One of the more interesting recorders is the HDF5 recorder: 
 [`sardana.macroserver.recorders.h5storage.py`](https://github.com/sardana-org/sardana/tree/develop/src/sardana/macroserver/recorders/h5storage.py)
 


 

### Macros: Custom Recorders

### Simple Example



* The following code, shows a basic example of a Custom Scan Recorder:

[`sardana/macroserver/recorders/examples/dummy.py`](https://github.com/sardana-org/sardana/tree/develop/src/sardana/macroserver/recorders/examples/dummy.py)

This dummy recorder, prints the output data in the MacroServer, and it prints different variables related to the launched Scan.
 


### Macros: Custom Recorders

### Real Application

* BL04 @ ALBA

* **madrecorder** is a Custom Recorder used for recording data from MAD detector in BL04 at ALBA.

